In [15]:
import numpy as np
import pynmea2
import pandas as pd
import struct
from datetime import datetime, timedelta #for handleing time
import re  #for string parcing
import os #for path search

In [16]:
folder_raw_data='data_raw'
filename_raw_data='Medicion_manual_test' #Cambiar por el nombre del archivo que se quiera convertir a csv
filename_folder_data_out='data_csv'
filename= folder_raw_data+'/'+filename_raw_data+'.DAT'
gps_garmin_hdop_circle_size=5 #change to make the visual part according
gps_decimal_to_meters=111139#convert decimal to meters. 111139 meters equals 1.0 lat/lon value in decimal
gps_garmin_lat_error=1/gps_decimal_to_meters #Convert decimal to meters, uANDRES:added for completing the code, i do not know the real value of this
gps_garmin_lon_error=1/gps_decimal_to_meters #:added for completing the code, add the real value

In [20]:
UNSIGNED_INT='<I' #4 BYTE UNSIGNED INT
SIGNED_INT='<i' #4 BYTE UNSIGNED INT
CHAR_TASK_NAME='12s' #4 BYTE UNSIGNED INT
CHAR_TASK_NAME_OFFSET=2
CHAR_TASK_NAME_LEN=12
CHAR_TASK_NAME_BOOTUP_WRITE=b'SYSTEM INFO' #4 BYTE UNSIGNED INT
DATA_SIZE_OFFSET=14
DATA_OFFSET=16
DATA_TIME_RES_OFFSET=244 #V1.0 is this value. Version above: the value is specified in the .dat file
RTOS_time_resolutionHz=1000 #default value
#get time value
TIME_OFFSET=20 #lenght=4 byutes

GYRO_START=50#50 is the value

dt = np.dtype(np.uint8)      # 8-bit unsigned integer
# reminder:CR=13
#reminderLF=10
time_start=0;
start_time_raw=0
end_time_raw=0
np.set_printoptions(precision=3) #print 3 digits only
acc_scaler= 0.00059841 #scale 0.000061=1g, 0,00059841=1 m/s
gyro_scaler= 0.00875   #scale 0.00875 for +-245 degree

#global variables
global float_res_time;
float_res_time=3;#default value
#return the indexes in which an structure begin
def get_time_bootup(byte_array):
      fr_version=bytes.decode((byte_array[215:218]).tobytes(), 'ascii')
      #print(fr_version)
      if fr_version=='1.0':
          RTOS_time_resolutionHz=bytes_to_32uint(byte_array[244:244+4])
          dateTime_rtos_offset_secs=(bytes_to_32uint(byte_array[260:260+4])/RTOS_time_resolutionHz)
          str_toparse_dateTimeUTC=bytes.decode((byte_array[110:180]).tobytes(), 'ascii')
          str_aux=str_toparse_dateTimeUTC.split('date:')[1]
          str_aux_date=str_aux.split(' ')[0]
          str_aux2=str_toparse_dateTimeUTC.split('time ')[1]
          str_aux_time=str_aux2.split(' UTC')[0]
          #get date time
          dateTime_UTC_str=str_aux[0:4]+'-'+str_aux[4:6]+'-'+str_aux[6:8]+' '+str_aux_time
      else:#version 1.1 and default
          RTOS_time_resolutionHz=bytes_to_32uint(byte_array[244:244+4]) #same as 1.0
          dateTime_rtos_offset_secs=(bytes_to_32uint(byte_array[268:268+4])/RTOS_time_resolutionHz)
          #print("date time offset",dateTime_rtos_offset_secs)
          str_toparse_dateTimeUTC=bytes.decode((byte_array[110:180]).tobytes(), 'ascii')
          str_aux=str_toparse_dateTimeUTC.split('date:')[1]
          dateTime_UTC_str=str_aux.split(' UTC')[0]
      #print(dateTime_UTC_str)
      dateTime_UTC=datetime.strptime(dateTime_UTC_str, '%Y-%m-%d %H:%M:%S')#does not include decimals
      #print(dateTime_UTC)
      return RTOS_time_resolutionHz,dateTime_UTC,dateTime_rtos_offset_secs
def get_accelGyro(byte_array,dateTime_UTC,time_start,time_end):
      global float_res_time;
      gx=[];gy=[];gz=[]; ax=[];ay=[];az=[];a_time=[];
      data_written=bytes_to_16uint(byte_array[DATA_OFFSET:DATA_OFFSET+2])
      time_res=(time_end-time_start)/data_written
      float_digitsg=2
      float_digitsa=4
      f_res_time=-6+4
      gyro_on=bytes_to_16uint(byte_array[34:36]) #DEFINED
      #print(time_start)
     ## print(time_res)
      if gyro_on:
        for i in range(data_written):
            c_pos=GYRO_START+12*i
            gx.append(round(bytes_to_gyro(byte_array[c_pos:      c_pos+2]),float_digitsg))
            gy.append(round(bytes_to_gyro(byte_array[c_pos+2:  c_pos+2+2]),float_digitsg))
            gz.append(round(bytes_to_gyro(byte_array[c_pos+4:  c_pos+4+2]),float_digitsg))
            ax.append(round(bytes_to_acc(byte_array[c_pos+6:  c_pos+6+2]),float_digitsa))
            ay.append(round(bytes_to_acc(byte_array[c_pos+8:  c_pos+8+2]),float_digitsa))
            az.append(round(bytes_to_acc( byte_array[c_pos+10:c_pos+10+2]),float_digitsa))
            a_time_add= dateTime_UTC+ timedelta(seconds=time_start+i*time_res)
            a_time.append(a_time_add.strftime("%Y-%m-%d %H:%M:%S.%f")[:f_res_time])
      else:
        for i in range(data_written):
            c_pos=GYRO_START+6*i
            gx.append(0) #append a 0 to maintaain consystency
            gy.append(0)
            gz.append(0)
            ax.append(round(bytes_to_acc(byte_array[c_pos:    c_pos+2]),float_digitsa))
            ay.append(round(bytes_to_acc(byte_array[c_pos+2:c_pos+2+2]),float_digitsa))
            az.append(round(bytes_to_acc(byte_array[c_pos+4:c_pos+4+2]),float_digitsa))
            a_time_add= dateTime_UTC+ timedelta(seconds=time_start+i*time_res)
            a_time.append(a_time_add.strftime("%Y-%m-%d %H:%M:%S.%f")[:f_res_time])
      #print(a_time)
      return gx,gy,gz,ax,ay,az,a_time

#auxiliwary functions
def bytes_to_float(byte_array):
    return(struct.unpack('f', byte_array) [0])
def bytes_to_32uint(byte_array):
     return struct.unpack(UNSIGNED_INT, byte_array)[0]
     # return (byte_array[0]+256*byte_array[1]+256*byte_array[1]+65534*byte_array[2]+16777216*byte_array[3])
def bytes_to_16uint(byte_array):
     return(byte_array[0]+256*byte_array[1])
def bytes_to_16int(byte_array):
     signed=bytes_to_16uint(byte_array)
     if signed>32767: signed=signed-65536
     return(signed)
def bytes_to_8int(byte_array):
      signed=byte_array[0]
      if signed>127: signed=signed-255
      return(signed)
def bytes_to_mag(byte_array):
      return(mag_scaler*bytes_to_16int(byte_array))
def bytes_to_acc(byte_array):
      return(acc_scaler*bytes_to_16int(byte_array))
def bytes_to_gyro(byte_array):
      return(gyro_scaler*bytes_to_16int(byte_array))
def find_start_indexes(byte_array):
    index_with_lines_br = np. where((byte_array == 13 ))[0]
    intex_ok=[]
    for index in index_with_lines_br:
        #print(index)
        if ((byte_array[index+1]==10)and((123>byte_array[index+2]>47)or (byte_array[index+2]==33))):
            intex_ok.append(index)
    return intex_ok


def get_datacsv_from_file(byte_array,intexes_ok):
     global float_res_time;
     lat=[];lon=[];vdop=[];hdop=[];pdop=[];time_gps=[];time_gps_to_fix=[];
     gx=[];gy=[];gz=[];ax=[];ay=[];az=[];a_time=[];
     mx=[];my=[];mz=[];m_time=[];
     t_mcu=[];t_imu=[];t_time=[];
     animal_activity=[];animal_time=[];
     time_start=0;
     start_time_raw=0
     end_time_raw=0
     for index in intexes_ok:
       task_name=struct.unpack(CHAR_TASK_NAME, byte_array[index+CHAR_TASK_NAME_OFFSET:index+CHAR_TASK_NAME_OFFSET+CHAR_TASK_NAME_LEN]) [0]
       #print(task_name[0:12])
       if task_name[0:len(CHAR_TASK_NAME_BOOTUP_WRITE)]==CHAR_TASK_NAME_BOOTUP_WRITE:#detect if a write has happened
           #GET OPERATIVE SYSTEM TIME RESOLUTION AND DATE FOR FUTURE READINGS.
           RTOS_time_resolutionHz,dateTime_UTC,dateTime_rtos_offset_secs=get_time_bootup(byte_array[index:index+300])
           if (RTOS_time_resolutionHz<1001): float_res_time=3 #ms resolution
           else:                                              float_res_time=6 #us resolution
           time_scaler=round(1/RTOS_time_resolutionHz,float_res_time) #convert hz to sec with desired resolution
           #print("first write time resolution in ms:",time_scaler)
           time_start=start_time_raw#Error: No such resource: /ti/utils/runtime/ExecGraphAdapter.syscfg.js#fix using the last started time
           #print("System boot up at:",dateTime_UTC)
           if len(animal_time)==0:
                animal_time.append(dateTime_UTC.strftime("%Y-%m-%d %H:%M:%S"))
                animal_activity.append(0)
       else:
           #parse as timestamp of sensor data: this means that
           data_size=bytes_to_16uint(byte_array[index+DATA_SIZE_OFFSET:index+DATA_SIZE_OFFSET+2])
           data_written=bytes_to_16uint(byte_array[index+DATA_OFFSET:index+DATA_OFFSET+2])
           # -dateTime_rtos_offset_secs is the RTOS time in which the dateTime_UTC was stored.
           start_time_raw=time_start-dateTime_rtos_offset_secs+struct.unpack(UNSIGNED_INT, byte_array[index+TIME_OFFSET:index+TIME_OFFSET+4])[0] #convert 4 bytes to unisgned int
           end_time_raw=time_start-dateTime_rtos_offset_secs+struct.unpack(UNSIGNED_INT, byte_array[index+TIME_OFFSET+4:index+TIME_OFFSET+8])[0] #convert 4 bytes to unisgned int
           start_time_offset=start_time_raw*time_scaler
           end_time_offset=end_time_raw*time_scaler
    #start to parse the packet
       if(task_name[0:6]==b'IMU0AG'):##accelerometer/gyro
            gxu,gyu,gzu,axu,ayu,azu,a_timeu=get_accelGyro(byte_array[index:index+data_size],dateTime_UTC,start_time_offset,end_time_offset)
            gx=gx+(gxu);gy=gy+(gyu);gz=gz+(gzu);ax=ax+(axu);ay=ay+(ayu);az=az+(azu);a_time=a_time+(a_timeu);
       
    #----------accel gyro
     data=np.concatenate([np.array(a_time),np.asarray(ax),np.asarray(ay),np.asarray(az),np.asarray(gx),np.asarray(gy),np.asarray(gz)])
     data=data.reshape(7,(int(len(data)/7)))
     accel_frame= pd.DataFrame(data.T,columns=['dateTime_UTC','ax', 'ay', 'az','gx', 'gy', 'gz'])
     
     data=np.concatenate([np.array(animal_time),np.asarray(animal_activity)]);
     data=data.reshape(2,(int(len(data)/2)))
     return accel_frame


byte_array = np.fromfile(filename, dtype=dt)

#print("file size in bytes:",byte_array.size)
intexes_ok=find_start_indexes(byte_array)
#print("indices ok",intexes_ok[0:10])
accel_frame=get_datacsv_from_file(byte_array,intexes_ok)
#export to csv
#-------------------------------------------------------------------
#check if output dir exist, otherwise, create it
if not os.path.exists(filename_folder_data_out):
    os.makedirs(filename_folder_data_out)

filename_data_out=filename_folder_data_out+'/'+filename_raw_data

accel_frame.to_csv(filename_data_out+'_accelGyro.csv',index=False)


In [21]:
import plotly.graph_objects as go
import pandas as pd

filename_data_out=filename_folder_data_out+'/'+filename_raw_data
accel_frame=pd.read_csv(filename_data_out+'_accelGyro.csv')

#-----------------accel
figa=go.Figure()#plot acceleration in ratio 9.81 m/s*2 =1
figa.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['ax'].values[:],name="ax",line_width=3))#px.scatter(accel_frame, x="time", y="ax")
figa.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['ay'].values[:],name="ay",line_width=3))
figa.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['az'].values[:],name="az",line_width=3))
figa.update_layout(title="Accelemerometer meassures ",xaxis_title="Time [Seconds]",yaxis_title="Accelemerometer axis value [m/s²]",legend_xanchor="right")
figa.show()
#-----------------gyro
figg=go.Figure()#plot gyroscope in degree/s
figg.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['gx'].values[:],name="gx",line_width=3))#px.scatter(accel_frame, x="time", y="ax")
figg.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['gy'].values[:],name="gy",line_width=3))
figg.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['gz'].values[:],name="gz",line_width=3))
figg.update_layout(title="Gyroscope meassures ",xaxis_title="Time [Seconds]",yaxis_title="Gyroscope axis value [°/s]",legend_xanchor="right")
figg.show()

